In [1]:
!pwd

/Users/johnson.huang/master_program/UMD/2024_Fall/DATA602/HW/final_project_602


# read data

## overview data

In [59]:
import pandas as pd


df_overv = pd.read_excel('data/Overview.xlsx', engine='openpyxl')
df_overv.rename(columns = {'Unnamed: 8': 'soc difference'}, inplace = True)
df_overv = df_overv.drop(['Unnamed: 13', 'Note'], axis=1)
print(f'original shape: {df_overv.shape}')
df_overv.head(3)

original shape: (73, 14)


,Trip,Date,Route/Area,Weather,Battery Temperature (Start) [°C],Battery Temperature (End),Battery State of Charge (Start),Battery State of Charge (End),soc difference,Ambient Temperature (Start) [°C],Target Cabin Temperature,Distance [km],Duration [min],Fan
0,TripA01,2019-06-25_13-21-14,Munich East,sunny,21.0,22.0,0.863,0.803,0.060,25.5,23.0,7.427690,16.82,"Automatic, Level 1"
1,TripA02,2019-06-25_14-05-31,Munich East,sunny,23.0,26.0,0.803,0.673,0.130,32.0,23.0,23.509709,23.55,"Automatic, Level 1"
2,TripA03,2019-06-28_10-02-15,Munich East,sunny,24.0,25.0,0.835,0.751,0.084,21.5,27.0,12.820846,11.18,"Automatic, Level 1"


### basic preprocessing

#### column name

In [60]:
# Create a dictionary to map old column names to new ones
column_mapping = {col: col.replace(',', '').replace('[', '').replace(']', '').replace('>', '') for col in df_overv.columns}  # to prevent 'ValueError: feature_names must be string, and may not contain [, ] or <' in xgboost

# Rename the columns using the mapping
df_overv.rename(columns=column_mapping, inplace=True)

#### nan

In [61]:
# discard nans
df_overv.dropna(how="any", inplace=True)
df_overv.reset_index(drop=True, inplace=True)
print(f'after drop nans shape: {df_overv.shape}')

after drop nans shape: (70, 14)


In [62]:
print(list(df_overv.columns))

['Trip', 'Date', 'Route/Area', 'Weather', 'Battery Temperature (Start) °C', 'Battery Temperature (End)', 'Battery State of Charge (Start)', 'Battery State of Charge (End)', 'soc difference', 'Ambient Temperature (Start) °C', 'Target Cabin Temperature', 'Distance km', 'Duration min', 'Fan']


In [63]:
df_overv.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 70 entries, 0 to 69
Data columns (total 14 columns):
 #   Column                           Non-Null Count  Dtype  
---  ------                           --------------  -----  
 0   Trip                             70 non-null     object 
 1   Date                             70 non-null     object 
 2   Route/Area                       70 non-null     object 
 3   Weather                          70 non-null     object 
 4   Battery Temperature (Start) °C   70 non-null     float64
 5   Battery Temperature (End)        70 non-null     float64
 6   Battery State of Charge (Start)  70 non-null     float64
 7   Battery State of Charge (End)    70 non-null     float64
 8   soc difference                   70 non-null     float64
 9   Ambient Temperature (Start) °C   70 non-null     float64
 10  Target Cabin Temperature         70 non-null     float64
 11  Distance km                      70 non-null     float64
 12  Duration min            

#### one hot for categorical

In [64]:
# Identify columns with object (string) dtype
object_cols = df_overv.select_dtypes(include=['object']).columns
# Apply one-hot encoding to these columns
df_overv = pd.get_dummies(df_overv, columns=object_cols)
print(f'after one hot - shape: {df_overv.shape}')
print(f'columns: {list(df_overv.columns)}')
df_overv.head(3)

after one hot - shape: (70, 167)
columns: ['Battery Temperature (Start) °C', 'Battery Temperature (End)', 'Battery State of Charge (Start)', 'Battery State of Charge (End)', 'soc difference', 'Ambient Temperature (Start) °C', 'Target Cabin Temperature', 'Distance km', 'Duration min', 'Trip_TripA01', 'Trip_TripA02', 'Trip_TripA03', 'Trip_TripA04', 'Trip_TripA05', 'Trip_TripA06', 'Trip_TripA07', 'Trip_TripA08', 'Trip_TripA09', 'Trip_TripA10', 'Trip_TripA11', 'Trip_TripA12', 'Trip_TripA13', 'Trip_TripA14', 'Trip_TripA15', 'Trip_TripA16', 'Trip_TripA17', 'Trip_TripA18', 'Trip_TripA19', 'Trip_TripA20', 'Trip_TripA21', 'Trip_TripA22', 'Trip_TripA23', 'Trip_TripA24', 'Trip_TripA25', 'Trip_TripA26', 'Trip_TripA27', 'Trip_TripA28', 'Trip_TripA29', 'Trip_TripA30', 'Trip_TripA31', 'Trip_TripA32', 'Trip_TripB01', 'Trip_TripB02', 'Trip_TripB03', 'Trip_TripB04', 'Trip_TripB05', 'Trip_TripB06', 'Trip_TripB07', 'Trip_TripB08', 'Trip_TripB09', 'Trip_TripB10', 'Trip_TripB11', 'Trip_TripB12', 'Trip_TripB

,Battery Temperature (Start) °C,Battery Temperature (End),Battery State of Charge (Start),Battery State of Charge (End),soc difference,Ambient Temperature (Start) °C,Target Cabin Temperature,Distance km,Duration min,Trip_TripA01,...,Weather_cloudy,Weather_dark,"Weather_dark, little rainy",Weather_rainy,Weather_slightly cloudy,Weather_sunny,Weather_sunrise,Weather_sunset,"Fan_Automatic, Level 1","Fan_Automatic, Level 1 (screen defrost for the first 2 min)"
0,21.0,22.0,0.863,0.803,0.060,25.5,23.0,7.427690,16.82,1,...,0,0,0,0,0,1,0,0,1,0
1,23.0,26.0,0.803,0.673,0.130,32.0,23.0,23.509709,23.55,0,...,0,0,0,0,0,1,0,0,1,0
2,24.0,25.0,0.835,0.751,0.084,21.5,27.0,12.820846,11.18,0,...,0,0,0,0,0,1,0,0,1,0


#### define X, y

In [65]:
# Assuming 'Sales' is the target variable
X_overv = df_overv.drop(['soc difference', 'Battery State of Charge (Start)', 'Battery State of Charge (End)'], axis=1)
y_overv = df_overv['soc difference']

#### train test split

In [66]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(
    X_overv,
    y_overv,
    test_size=0.2,
    random_state=30, # set random seed for reproducibility
)

### tree based baseline model

In [67]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier, RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier as DTC
from sklearn.tree import plot_tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.inspection import permutation_importance

#### decision tree

In [68]:
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
import numpy as np

# Create and train the model
dt_regressor = DecisionTreeRegressor(random_state=42)
dt_regressor.fit(X_train, y_train)

# Make predictions
y_pred = dt_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Decision Tree RMSE: {rmse}")

Decision Tree RMSE: 0.18497818404187175


#### random forest

In [69]:
from sklearn.ensemble import RandomForestRegressor

# Create and train the model
rf_regressor = RandomForestRegressor(n_estimators=100, random_state=30)
rf_regressor.fit(X_train, y_train)

# Make predictions
y_pred = rf_regressor.predict(X_test)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"Random Forest RMSE: {rmse}")

Random Forest RMSE: 0.17240375198933464


#### xgboost

In [70]:
import xgboost as xgb

# Create DMatrix for XGBoost
dtrain = xgb.DMatrix(X_train, label=y_train)
dtest = xgb.DMatrix(X_test, label=y_test)

# Set parameters
params = {
    'max_depth': 5,
    'eta': 0.3,
    'objective': 'reg:squarederror',
    'eval_metric': 'rmse'
}

# Train the model
num_round = 100
xgb_model = xgb.train(params, dtrain, num_round)

# Make predictions
y_pred = xgb_model.predict(dtest)

# Evaluate the model
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
print(f"XGBoost RMSE: {rmse}")

XGBoost RMSE: 0.18081997928970778


## trip data

### perprocess - merge all files for the first time

In [10]:
# Load in the dataset
import os
import glob
import chardet
import pandas as pd


df_master = pd.DataFrame()
for file_name in glob.glob('data/'+'*.csv'):
    rawdata = open(file_name, 'rb').read()
    result = chardet.detect(rawdata)
    charenc = result['encoding']
    df_trip = pd.read_csv(file_name, sep = ';',encoding = charenc)
    df_master = df_master.append(df_trip)
    print(f'finished processing: {file_name}')

df_master.head()

finished processing: data/TripA27.csv
finished processing: data/TripA26.csv
finished processing: data/TripA32.csv
finished processing: data/TripA18.csv
finished processing: data/TripA24.csv
finished processing: data/TripA30.csv
finished processing: data/TripA31.csv
finished processing: data/TripA25.csv
finished processing: data/TripA19.csv
finished processing: data/TripA21.csv
finished processing: data/TripA09.csv
finished processing: data/TripA08.csv
finished processing: data/TripA20.csv
finished processing: data/TripA22.csv
finished processing: data/TripA23.csv
finished processing: data/TripB26.csv
finished processing: data/TripB32.csv
finished processing: data/TripB33.csv
finished processing: data/TripB27.csv
finished processing: data/TripB19.csv
finished processing: data/TripB31.csv
finished processing: data/TripB25.csv
finished processing: data/TripB24.csv
finished processing: data/TripB30.csv
finished processing: data/TripB18.csv
finished processing: data/TripB34.csv
finished pro

,Time [s],Velocity [km/h],Elevation [m],Throttle [%],Motor Torque [Nm],Longitudinal Acceleration [m/s^2],Regenerative Braking Signal,Battery Voltage [V],Battery Current [A],Battery Temperature [°C],...,Temperature Feetvent Co-Driver [°C],Temperature Feetvent Driver [°C],Temperature Head Co-Driver [°C],Temperature Head Driver [°C],Temperature Vent right [°C],Temperature Vent central right [°C],Temperature Vent central left [°C],Temperature Vent right [°C],Velocity [km/h]]],Unnamed: 23
0,0.0,0.0,521.0,0.0,0.0,-0.10800,0.0,388.1,-1.30000,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0.1,0.0,521.0,0.0,0.0,-0.09862,0.0,388.1,-1.30000,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0.2,0.0,521.0,0.0,0.0,-0.11609,0.0,388.1,-1.29807,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0.3,0.0,521.0,0.0,0.0,-0.12928,0.0,388.1,-1.24808,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0.4,0.0,521.0,0.0,0.0,-0.10800,0.0,388.1,-1.20191,20.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
# Remove last 2 columns since they are Nan
df_master = df_master.iloc[: , :-2]
print(list(df_master.columns.values))

# # output as a file
# df_master.to_csv('data/all_trips.csv', index=False)

['Time [s]', 'Velocity [km/h]', 'Elevation [m]', 'Throttle [%]', 'Motor Torque [Nm]', 'Longitudinal Acceleration [m/s^2]', 'Regenerative Braking Signal ', 'Battery Voltage [V]', 'Battery Current [A]', 'Battery Temperature [°C]', 'max. Battery Temperature [°C]', 'SoC [%]', 'displayed SoC [%]', 'min. SoC [%]', 'max. SoC [%)', 'Heating Power CAN [kW]', 'Heating Power LIN [W]', 'Requested Heating Power [W]', 'AirCon Power [kW]', 'Heater Signal', 'Heater Voltage [V]', 'Heater Current [A]', 'Ambient Temperature [°C]', 'Coolant Temperature Heatercore [°C]', 'Requested Coolant Temperature [°C]', 'Coolant Temperature Inlet [°C]', 'Heat Exchanger Temperature [°C]', 'Cabin Temperature Sensor [°C]', 'Ambient Temperature Sensor [°C]', 'Coolant Volume Flow +500 [l/h]', 'Temperature Coolant Heater Inlet [°C]', 'Temperature Coolant Heater Outlet [°C]', 'Temperature Heat Exchanger Outlet [°C]', 'Temperature Defrost lateral left [°C]', 'Temperature Defrost lateral right [°C]', 'Temperature Defrost centr